In [1]:
# driveのマウント
from google.colab import drive
drive.mount('/content/drive')

# モデルのバージョン
VER = '005'

Mounted at /content/drive


# ライブラリのインポート

In [2]:
import gc
import os
import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

!pip install japanize-matplotlib
import japanize_matplotlib
import folium


from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.1 MB 31.1 MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120275 sha256=94a470b1cf0e6af28923e8ba256a5f7a91686adac4a1877f9912fc493676dc6a
  Stored in directory: /root/.cache/pip/wheels/4f/ca/96/4cc5e192421cceb077fbf4ffec533382edd416fd3fa0af0bbd
Successfully built japanize-matplotlib


# データのロード

In [3]:
# パスは適宜変更してください

# ディレクトリを移動
os.chdir('/content/drive/MyDrive/signate-908-hiroshima/')
path = os.getcwd()

# データ読み込み
water_data = pd.read_csv(os.path.join(path, 'train', 'waterlevel', 'data.csv'))
water_stations = pd.read_csv(os.path.join(path, 'train', 'waterlevel', 'stations.csv'))
# rain_data = pd.read_csv(os.path.join(path, 'train', 'rainfall', 'data.csv'))
# rain_stations = pd.read_csv(os.path.join(path, 'train', 'rainfall', 'stations.csv'))
# tide_data = pd.read_csv(os.path.join(path, 'train', 'tidelevel', 'data.csv'))
# tide_stations = pd.read_csv(os.path.join(path, 'train', 'tidelevel', 'stations.csv'))

# データ前処理

In [4]:
def preprocess_water_data_station(water_data, water_stations):
    """water_dataのstationの前処理を行う
    """
    # 欠損値補完
    water_data['river'] = water_data['river'].replace('\u3000', '沼田川')

    # (国)への変更前観測所名を変換
    stations = water_data.loc[water_data['station'].str.contains(r'\(国\)'), 'station'].unique()
    # 中野、伊尾、和木は(国)を含まない観測所が別途存在するため別処理
    stations = [x.replace('(国)', '') for x in stations if x not in ['中野(国)', '伊尾(国)', '和木(国)']]
    water_data['station'] = water_data['station'].apply(lambda x: x + '(国)' if x in stations else x)
    # 中野、伊尾、和木は河川名で分けて処理
    water_data.loc[(water_data['station']=='中野')&(water_data['river']=='太田川'), 'station'] = '中野(国)'
    water_data.loc[(water_data['station']=='伊尾')&(water_data['river']=='芦田川'), 'station'] = '伊尾(国)'
    water_data.loc[(water_data['station']=='和木')&(water_data['river']=='小瀬川'), 'station'] = '和木(国)'

    # (電)への変更前観測所名を変換
    stations = water_data.loc[water_data['station'].str.contains(r'\(電\)'), 'station'].unique()
    stations = [x.replace('(電)', '') for x in stations]
    water_data['station'] = water_data['station'].apply(lambda x: x + '(電)' if x in stations else x)

    # 入力ミスと思われるもの
    water_data['station'] = water_data['station'].replace({'藤波': '藤浪',
                                                           '中州橋': '中洲橋',
                                                           '段原': '段原(猿猴川)'})
    water_data['river'] = water_data['river'].replace({'手越川': '手城川',
                                                       '横川': '横川川'})

    # 入力されない観測所を削除
    in_stations = water_stations.loc[water_stations['入力時使用']==1, '観測所名称'].unique()
    water_data = water_data[water_data['station'].isin(in_stations)]

    return water_data


def preprocess_rain_data_station(rain_data):
    """rain_dataの前処理
    """
    # 重複行を削除
    rain_data = rain_data.drop_duplicates()

    # 2191日分のデータがある観測所のみ使用する
    stations = rain_data.groupby(['station', 'city']).count()['date'].reset_index()
    stations = stations.loc[stations['date']==2191, 'station'].unique()
    rain_data = rain_data[rain_data['station'].isin(stations)]

    return rain_data

In [5]:
# 水位データの観測所をクリーニング
water_data = preprocess_water_data_station(water_data, water_stations)
# rain_data = preprocess_rain_data_station(rain_data)

In [6]:
# 水位データを入力形式に整形
# ほぼrun.pyからコピペ
stations = set(water_stations[water_stations['評価対象']==1]['観測所名称'])

in_all_data = {}

print('processing water data')
for data in tqdm(water_data.groupby('date')):
    day = data[0]
    data_dict = data[1].to_dict('records')
    in_data = []
    for d in data_dict:
        for k in d.keys():
            if k not in ('date', 'station', 'river'):
                in_data.append({'station':d['station'], 'river':d['river'], 'hour':int(k.split(':')[0]), 'value':d[k]})
    in_all_data[day] = {}
    in_all_data[day]['date'] = day
    in_all_data[day]['stations'] = stations
    in_all_data[day]['waterlevel'] = in_data

print('done')
# print('processing rain data')

# for data in tqdm(rain_data.groupby('date')):
#     day = data[0]
#     data_dict = data[1].to_dict('records')
#     in_data = []
#     for d in data_dict:
#         for k in d.keys():
#             if k not in ('date', 'station', 'city'):
#                 in_data.append({'station':d['station'], 'city':d['city'], 'hour':int(k.split(':')[0]), 'value':d[k]})
#     in_all_data[day]['rainfall'] = in_data

# print('done')


processing water data


100%|██████████| 2191/2191 [00:31<00:00, 69.20it/s]

done


In [7]:
# 入力データをdfに変換
water_df = []
# rain_df = []
for d in tqdm(range(len(in_all_data))):
    tmp = pd.DataFrame(in_all_data[d]['waterlevel'])
    tmp['date'] = d
    water_df.append(tmp)

    # tmp = pd.DataFrame(in_all_data[d]['rainfall'])
    # tmp['date'] = d
    # rain_df.append(tmp)

water_df = pd.concat(water_df)
# rain_df = pd.concat(rain_df)

100%|██████████| 2191/2191 [00:10<00:00, 206.71it/s]


In [8]:
# 水位データを数値に変換
water_df['value'] = pd.to_numeric(water_df['value'], errors='coerce')
# rain_df['value'] = pd.to_numeric(rain_df['value'], errors='coerce')
# rain_df.isna().sum()

# 水系名を追加
water_df = water_df.merge(water_stations[['観測所名称', '河川名', '水系名']], left_on=['station', 'river'], right_on=['観測所名称', '河川名'], how='left').drop(['観測所名称', '河川名'], axis=1)

In [9]:
# # 水位観測所のプロット
# # systems = water_stations['水系名'].unique()
# systems = ['江の川', '太田川']

# map = folium.Map(location=['34.5447', '132.8134'], zoom_start=10, tiles='OpenStreetMap')
# water_station = water_stations[water_stations['水系名'].isin(systems)].dropna(subset=['緯度', '経度'])

# for i in range(len(water_station)):
#     folium.Circle(radius=150, location=[water_station.iloc[i]['緯度'], water_station.iloc[i]['経度']],
#                   tooltip=str(water_station.iloc[i]['観測所名称'] + ' (' + water_station.iloc[i]['水系名'] + ')'),
#                   color='red', fill=False).add_to(map)

# map

In [10]:
# # 観測所ごとに1日の平均をプロット
# fig, axes = plt.subplots(90, 2, figsize=(20, 270), tight_layout=True)
# water_df = water_df.sort_values('水系名')
# for i, station in enumerate(tqdm(water_df['station'].unique())):
#     ax = axes[i//2, i%2]
#     df = water_df[water_df['station']==station]
#     river = df.iloc[0]['river']
#     system = df.iloc[0]['水系名']
#     df.groupby('date').mean()['value'].plot(ax=ax)
#     ax.set_xlim(0, 2200)
#     ax.set_title(f'観測所: {station} / 河川: {river} / 水系: {system} / NaNの比率: {round(df["value"].isna().sum()/len(df)*100, 2)}%')
# fig.show()

In [11]:
# 欠損値を線形補完
dfs = []
for group in tqdm(water_df.groupby('station')):
    df = group[1]
    dfs.append(df.interpolate())
water_df = pd.concat(dfs)

# 観測所ごとに1日の平均をプロット
# fig, axes = plt.subplots(90, 2, figsize=(20, 270), tight_layout=True)
# for i, station in enumerate(tqdm(water_df['station'].unique())):
#     ax = axes[i//2, i%2]
#     df = water_df[water_df['station']==station]
#     df.groupby('date').mean()['value'].plot(ax=ax)
#     ax.set_xlim(0, 2200)
#     ax.set_title(f'{station} / NaNの比率: {round(df["value"].isna().sum()/len(df)*100, 2)}%')
# fig.show()

100%|██████████| 178/178 [00:01<00:00, 120.17it/s]


In [12]:
# 不要な変数を削除してメモリを解放
# del water_data, water_stations, rain_data, rain_stations, stations, in_all_data, data_dict, in_data, df, dfs, tmp
# gc.collect()

In [13]:
# ワイドフォーマットに変更
# rain_df = rain_df[['station', 'date', 'hour', 'value']].pivot_table(index=['date', 'hour'], columns='station', values='value').reset_index()

In [14]:
# メモリ削減のためにデータ型を変更
# water_df['value'] = water_df['value'].astype('float32')
# rain_df.iloc[:, 2:] = rain_df.iloc[:, 2:].astype('float32')

In [15]:
# 水位データと降水量データをマージ
# water_df = water_df.merge(rain_df, on=['date', 'hour'], how='left')

# del rain_df
# gc.collect()

# 特徴量エンジニアリング

In [16]:
# 目的変数とシフト特徴量を作る
dfs = []
for group in tqdm(water_df.groupby('station')):
    df = group[1]
    for h in [-24]:
        if h==-24:
            df['y'] = df['value'].shift(h)
        else:
            col = 'shift_' + str(h) + 'h'
            df[col] = df['value'].shift(h)
            col = 'diff_' + str(h) + 'h'
            df[col] = df['value'].diff(periods=h)
    dfs.append(df)
water_df = pd.concat(dfs, axis=0).reset_index(drop=True)

# del df, dfs
# gc.collect()

100%|██████████| 178/178 [00:01<00:00, 172.53it/s]


In [17]:
value_23h = water_df.loc[water_df['hour']==23, ['date', 'station', 'river', 'value']]
water_df = water_df.merge(value_23h, on=['date', 'station', 'river'], how='left', suffixes=('', '_23h'))
water_df['v-v23h'] = water_df['value'] - water_df['value_23h']

# モデリング

In [18]:
# ライブラリインポート
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [19]:
# ラベルエンコーディング
label_enc_col = ['station', 'river']
for col in label_enc_col:
    le = LabelEncoder()
    water_df[col] = le.fit_transform(water_df[col].values)

In [20]:
water_df.head()

,station,river,hour,value,date,水系名,y,value_23h,v-v23h
0,0,59,0,1.64,0,沼田川,1.64,1.64,0.0
1,0,59,1,1.64,0,沼田川,1.64,1.64,0.0
2,0,59,2,1.64,0,沼田川,1.64,1.64,0.0
3,0,59,3,1.64,0,沼田川,1.64,1.64,0.0
4,0,59,4,1.64,0,沼田川,1.64,1.64,0.0


In [21]:
# 特徴量カラム
features = ['date', 'hour', 'station', 'river', 'value_23h']

# train, valid, testに分ける
train = water_df.loc[water_df['date']<=1314]
valid = water_df.loc[(water_df['date']>1314)&(water_df['date']<=1752)]
test = water_df.loc[water_df['date']>1752]

# NaNを含む行を削除
train = train[features + ['y']].dropna()
valid = valid[features + ['y']].dropna()
test = test[features + ['y']].dropna()

X_train, y_train = train[features], train['y']
X_valid, y_valid = valid[features], valid['y']
X_test, y_test = test[features], test['y']

In [22]:
params = {'objective': 'regression',
         'random_state': 42,
         'boosting_type': 'gbdt',
         'n_estimators': 10000
         }
         
# fit_params = {'callbacks':[lgb.early_stopping(stopping_rounds=50)],
#               'eval_metric': 'rmse',
#               'eval_set': [(X_valid, y_valid)]}

verbose_eval = 1

model = lgb.LGBMRegressor(**params)
model.fit(X_train, y_train, 
          eval_metric='rmse',  # early_stoppingの評価指標(学習用の'metric'パラメータにも同じ指標が自動入力される)
          eval_set=[(X_valid, y_valid)],
          early_stopping_rounds=50,
          verbose=verbose_eval
          )

[1]	valid_0's rmse: 20.659	valid_0's l2: 426.795
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's rmse: 18.5986	valid_0's l2: 345.908
[3]	valid_0's rmse: 16.7462	valid_0's l2: 280.436
[4]	valid_0's rmse: 15.0812	valid_0's l2: 227.444
[5]	valid_0's rmse: 13.5852	valid_0's l2: 184.557
[6]	valid_0's rmse: 12.2414	valid_0's l2: 149.851
[7]	valid_0's rmse: 11.0349	valid_0's l2: 121.769
[8]	valid_0's rmse: 9.95235	valid_0's l2: 99.0492
[9]	valid_0's rmse: 8.98165	valid_0's l2: 80.67
[10]	valid_0's rmse: 8.11199	valid_0's l2: 65.8043
[11]	valid_0's rmse: 7.33365	valid_0's l2: 53.7824
[12]	valid_0's rmse: 6.63789	valid_0's l2: 44.0616
[13]	valid_0's rmse: 6.01693	valid_0's l2: 36.2034
[14]	valid_0's rmse: 5.46378	valid_0's l2: 29.8529
[15]	valid_0's rmse: 4.97201	valid_0's l2: 24.7209
[16]	valid_0's rmse: 4.53604	valid_0's l2: 20.5757
[17]	valid_0's rmse: 4.15072	valid_0's l2: 17.2285
[18]	valid_0's rmse: 3.81136	valid_0's l2: 14.5265
[19]	valid_0's rmse: 3.5137	vali

LGBMRegressor(n_estimators=10000, objective='regression', random_state=42)

In [23]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration_)

In [24]:
pd.DataFrame({
    'feature': features,
    'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

,feature,importance
4,value_23h,526
2,station,339
1,hour,289
3,river,221
0,date,185


In [25]:
print(f'RMSE test: {(np.sqrt(mean_squared_error(y_test, y_pred)))}')

RMSE test: 0.2481893913081291


# モデル保存

In [26]:
import pickle

In [27]:
model_file = 'baseline_lightgbm_' + VER + '.pkl'
model_path = os.path.join(path, 'models', model_file)
with open(model_path, 'wb') as f:
    pickle.dump(model, f)